In [26]:
import random
import math
import json
from prototype import find_entryIds, find_treeIds, find_meaningIds
from data_loader import load_jsons

mapare, all_inflected_forms, word_to_id_pos, id_to_word_pos, id_to_inflected_forms, entry_lexeme, tree_entry, relation, synonyms, context = load_jsons()

contexts_found = {}
test = {}
train = {}

for id in entry_lexeme.entry_lexeme:
    if len(entry_lexeme.entry_lexeme[id]) > 1:
        entry_ids = find_entryIds([id])
        tree_ids = find_treeIds(entry_ids)
        meaning_ids = find_meaningIds(tree_ids)

        for treeId in tree_ids:
            
            if len(context.find_context(treeId)) > 1:
                test[treeId] = []
                train[treeId] = context.find_context(treeId)
                rand_test = random.randint(1, math.floor(len(train[treeId]) / 2) + 1)
                
                for i in range(rand_test):
                    rand_index = random.randint(0, len(train[treeId]) - 1)
                    test[treeId].append(train[treeId][rand_index])
                    train[treeId].pop(rand_index)


import os
if not os.path.exists("model_testing"):
    os.makedirs("model_testing")

with open("model_testing/train.json", "w") as fisier_json:
        json.dump(train, fisier_json, indent=4, sort_keys=True)

with open("model_testing/test.json", "w") as fisier_json:
        json.dump(test, fisier_json, indent=4, sort_keys=True)


 [INFO] 2024-01-06 17:58:45,990 root:177 - Start loading needed data in memory!
 [INFO] 2024-01-06 17:58:45,994 data_loader:189 - Mapare file loaded.


 [INFO] 2024-01-06 17:58:53,641 data_loader:191 - All inflected forms file loaded.
 [INFO] 2024-01-06 17:58:54,217 data_loader:193 - Mapping word to id and pos file loaded.
 [INFO] 2024-01-06 17:58:54,791 data_loader:195 - Mapping word id to word and pos file loaded.
 [INFO] 2024-01-06 17:58:58,119 data_loader:197 - Mapping id to inflected forms file loaded.
 [INFO] 2024-01-06 17:59:00,725 data_loader:199 - Mapping entry id to lexeme id file loaded.
 [INFO] 2024-01-06 17:59:00,922 data_loader:201 - Mapping tree id to entry id file loaded.
 [INFO] 2024-01-06 17:59:00,973 data_loader:203 - Mapping meaning id to tree id file loaded.
 [INFO] 2024-01-06 17:59:01,128 data_loader:205 - Mapping synonyms file loaded.
 [INFO] 2024-01-06 17:59:01,596 data_loader:207 - Mapping contexts file loaded.
 [INFO] 2024-01-06 17:59:01,597 root:209 - The data from jsons files is now loaded in memory!


In [34]:
train = json.load(open("model_testing/train.json"))
test = json.load(open("model_testing/test.json"))

from transformers import AutoTokenizer, AutoModel
from scipy.spatial.distance import cosine
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import json

nltk.download('punkt')
nltk.download('stopwords')


def keep_letters_and_digits(sentence):
    sentence = sentence.lower()
    tokens = word_tokenize(sentence)
    tokens = [word for word in tokens if word.isalnum()]
    stop_words = set(stopwords.words('romanian'))
    tokens = [word for word in tokens if word not in stop_words]
    stemmer = SnowballStemmer("romanian")
    tokens = [stemmer.stem(word) for word in tokens]    
    return tokens

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

def calculate_context_similarity(sentence1, sentence2):
    emb1 = model.encode(sentence1)
    emb2 = model.encode(sentence2)
    cos_sim = util.cos_sim(emb1, emb2)
    return cos_sim

test_ctx_max_idx = 0
train_ctx_max_idx = 0

# evaluare provizorie:
# 44555 corect
# 39678 corect
# 6 wrong
# 9 wrong
# 196 wrong
# 255 wrong
# 44597 wrong
# 44644 corect

results = {}

for id in ["44644"]:
    if len(entry_lexeme.entry_lexeme[id]) > 1:
        entry_ids = find_entryIds([id])
        tree_ids = find_treeIds(entry_ids)
        tree_ids_test = find_treeIds(entry_ids)
        tree_ids_train = find_treeIds(entry_ids)

        if len(tree_ids) > 0:
            for treeId_test in tree_ids_test:
                max_similarity = 0

                for test_ctx in test[str(treeId_test)]:
                    print(test_ctx)
                    for treeId_train in tree_ids_train:
                        mean_similarity = []

                        for train_ctx in train[str(treeId_train)]:
                                        
                            actual_similarity = calculate_context_similarity(
                                            " ".join(keep_letters_and_digits(test_ctx)),
                                            " ".join(keep_letters_and_digits(train_ctx))
                                )
                            mean_similarity.append(actual_similarity)
                            
                            # provizoriu
                            try:
                                media = sum(mean_similarity) / len(mean_similarity)

                                if media > max_similarity:
                                        max_similarity = media
                                        max_test_ctx = test_ctx
                                        max_train_ctx = train_ctx
                                        aux = {str(treeId_train): (max_similarity, max_train_ctx, train_ctx_max_idx)}
                                        
                            except ZeroDivisionError:
                                print("pass")
                    
                results[str(treeId_test)] = aux
                print(results)        
                    
print("aici")
 
for key in results:
    for key2 in results[key]:
        print(key, ": ", key2, " => ", results[key][key2])
del results
# with open("model_testing/result.json", "w") as fisier_json:
        # json.dump(results, fisier_json, indent=4, sort_keys=True)


[nltk_data] Downloading package punkt to /Users/inttstbrd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/inttstbrd/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
 [INFO] 2024-01-06 18:03:31,671 sentence_transformers.SentenceTransformer:66 - Load pretrained SentenceTransformer: xlm-r-100langs-bert-base-nli-stsb-mean-tokens


 [INFO] 2024-01-06 18:03:37,309 sentence_transformers.SentenceTransformer:105 - Use pytorch device: cpu


Specie de uliu.


Batches: 100%|██████████| 1/1 [00:00<00:00, 19.52it/s]


{'282925': {'282925': (tensor([[0.7921]]), '@Porumb@ + sufix $-ar.$', 0)}}
Arbust sălbatic din familia rozaceelor, cu ramuri spinoase, cu flori albe și cu fructe sferice de culoare neagră-vineție, cu gust acru, astringent; porumbel[125759] $(Prunus spinosa).$


Batches: 100%|██████████| 1/1 [00:00<00:00, 22.36it/s]

{'282925': {'282925': (tensor([[0.7921]]), '@Porumb@ + sufix $-ar.$', 0)}, '44470': {'44470': (tensor([[0.4818]]), '@Porumbă[125746]@ + sufix $-ar.$', 0)}}
aici
282925 :  282925  =>  (tensor([[0.7921]]), '@Porumb@ + sufix $-ar.$', 0)
44470 :  44470  =>  (tensor([[0.4818]]), '@Porumbă[125746]@ + sufix $-ar.$', 0)


In [28]:
# prima slaba
# model_name = "dumitrescustefan/bert-base-romanian-cased-v1"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

# def calculate_context_similarity(input_sentence, sentences):
#     encoded_sentences = [tokenizer(sentence, return_tensors="pt") for sentence in sentences]
#     input_encoded = tokenizer(input_sentence, return_tensors="pt")

#     with torch.no_grad():
#         context_vectors = [model(**sentence)["last_hidden_state"][:, 0, :].squeeze() for sentence in encoded_sentences]
#         input_vector = model(**input_encoded)["last_hidden_state"][:, 0, :].squeeze()

#     similarities = [1 - cosine(input_vector.numpy(), context_vector.numpy()) for context_vector in context_vectors]
#     if len(similarities):
#         return (sum(similarities)/len(similarities))
#     else:
#         return 0
# 

# a doua parca mai ok
# model_name = 'bert-base-uncased'
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertModel.from_pretrained(model_name)

# def get_bert_embedding(sentence, model, tokenizer):
#     tokens = tokenizer(sentence, return_tensors='pt')
#     with torch.no_grad():
#         outputs = model(**tokens)
#     return outputs.pooler_output

# def calculate_context_similarity(sentence_test, sentence_train):
#     embedding_sentence_test = get_bert_embedding(sentence_test, model, tokenizer)
#     embedding_sentence_train = get_bert_embedding(sentence_train, model, tokenizer)

#     return 1 - cosine_similarity(embedding_sentence_test, embedding_sentence_train)[0][0]
    

# def cos_sim(input_vectors):
#     similarity = cosine_similarity(input_vectors)
#     return similarity

# # get cosine similairty matrix
# def calculate_context_similarity(sentence, sentences_list):
#     module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"

#     # Import the Universal Sentence Encoder's TF Hub module
#     embed = hub.load(module_url)

#     # Reduce logging output.
#     sentence_embedding = embed([sentence])
#     sentences_embeddings = embed(sentences_list)

#     # Calculate cosine similarity between the input sentence and the list of sentences
#     similarity_matrix = cos_sim(np.vstack([sentence_embedding, sentences_embeddings]))

#     # Extract similarity values for the input sentence compared to each sentence in the list
#     similarities = similarity_matrix[0, 1:]

#     max_similarity_index = np.argmax(similarities)

#     # Print the sentence with the maximum similarity
#     max_similarity_sentence = sentences_list[max_similarity_index]
#     max_similarity_value = similarities[max_similarity_index]

#     # print(f"Maximum similarity between '{sentence}' and '{max_similarity_sentence}': {max_similarity_value}")

#     return max_similarity_value
